In [2]:
import requests, json, pandas as pd, os
%pylab inline

from IPython.display import Image, HTML, display

Populating the interactive namespace from numpy and matplotlib


In [3]:
dhus_url = 'https://s5phub.copernicus.eu/dhus/search'
username = 's5pguest'
password = 's5pguest'

In [5]:
query = 'footprint:"Intersects(POLYGON((94.51952690411036 43.80793367773478,95.94731431958031 19.32621067474085,127.07307997682513 20.40046463519485,125.64529256135516 44.0136548172602,94.51952690411036 43.80793367773478,94.51952690411036 43.80793367773478)))" AND ( beginPosition:[2020-01-01T00:00:00.000Z TO 2020-03-21T23:59:59.999Z] AND endPosition:[2020-01-01T00:00:00.000Z TO 2020-03-21T23:59:59.999Z] ) AND ( (platformname:Sentinel-5 AND producttype:L2__NO2___ AND processingmode:Offline))'

In [46]:
payload = {
    'orderby': 'ingestiondate desc',
    'format': 'json',
    'rows': 100,
    'start': 0,
    'q': query,
}

response = requests.get(dhus_url, params=payload, auth=(username, password))
print(f'Reponse Stats: {response.status_code}')

Reponse Stats: 200


In [47]:
results = response.json()['feed']

if 'entry' not in results.keys():
    print('No Results')
    
else:
    print(f'Found: {len(results["entry"])}')

Found: 97


In [48]:
print(json.dumps(results['entry'][0], indent=2))

{
  "title": "S5P_OFFL_L2__NO2____20200205T050218_20200205T064349_11986_01_010302_20200208T100004",
  "link": [
    {
      "href": "https://s5phub.copernicus.eu/dhus/odata/v1/Products('7a3ade2d-77c3-42b3-bbaa-6cedcb144140')/$value"
    },
    {
      "rel": "alternative",
      "href": "https://s5phub.copernicus.eu/dhus/odata/v1/Products('7a3ade2d-77c3-42b3-bbaa-6cedcb144140')/"
    },
    {
      "rel": "icon",
      "href": "https://s5phub.copernicus.eu/dhus/odata/v1/Products('7a3ade2d-77c3-42b3-bbaa-6cedcb144140')/Products('Quicklook')/$value"
    }
  ],
  "id": "7a3ade2d-77c3-42b3-bbaa-6cedcb144140",
  "summary": "Date: 2020-02-05T05:23:53Z, Instrument: TROPOMI, Mode: , Satellite: Sentinel-5 Precursor, Size: 432.04 MB",
  "date": [
    {
      "name": "ingestiondate",
      "content": "2020-02-14T18:07:10.621Z"
    },
    {
      "name": "beginposition",
      "content": "2020-02-05T05:23:53Z"
    },
    {
      "name": "endposition",
      "content": "2020-02-05T06:22:16Z"
    }


In [49]:
result_info = []

for result in results['entry']:
    
    guid = result['id']
    data = [l['href'] for l in result['link'] if len(l.keys()) == 1][0]
    size = [l['content'] for l in result['str'] if l['name'] == 'size'][0]
    look = [l['href'] for l in result['link'] if 'Quicklook' in l['href']][0]
    igst = [d['content'] for d in result['date'] if d['name'] == 'ingestiondate'][0]
    
    result_info.append({
        'guid': guid,
        'data': data,
        'size': size,
        'look': look,
        'igst': igst,
    })
    
result_df = pd.DataFrame(result_info)

# Set Time Fields
result_df['igst'] = pd.to_datetime(result_df['igst'])
result_df.head()

,guid,data,size,look,igst
0,7a3ade2d-77c3-42b3-bbaa-6cedcb144140,https://s5phub.copernicus.eu/dhus/odata/v1/Pro...,432.04 MB,https://s5phub.copernicus.eu/dhus/odata/v1/Pro...,2020-02-14 18:07:10.621000+00:00
1,1472d6f3-081c-42a7-98c9-e657598d4dfb,https://s5phub.copernicus.eu/dhus/odata/v1/Pro...,431.71 MB,https://s5phub.copernicus.eu/dhus/odata/v1/Pro...,2020-02-14 18:06:40.710000+00:00
2,1698e399-a458-4765-b601-87c24aec4fc5,https://s5phub.copernicus.eu/dhus/odata/v1/Pro...,432.65 MB,https://s5phub.copernicus.eu/dhus/odata/v1/Pro...,2020-02-13 17:37:35.440000+00:00
3,d9dbf99e-a2c2-4be0-a5cd-f1e06b6932e6,https://s5phub.copernicus.eu/dhus/odata/v1/Pro...,431.46 MB,https://s5phub.copernicus.eu/dhus/odata/v1/Pro...,2020-02-13 17:37:34.950000+00:00
4,071f388b-f642-428d-ab06-3277405a7ee8,https://s5phub.copernicus.eu/dhus/odata/v1/Pro...,434.42 MB,https://s5phub.copernicus.eu/dhus/odata/v1/Pro...,2020-02-12 17:35:32.204000+00:00


In [50]:
output_dir = "/global/cscratch1/sd/yguan/data/s5p/"

In [51]:
def download_data(url):
    "download a given product given url"
    query = f'wget --content-disposition --continue '\
            f'--user={username} --password={password} '\
            f'-P {output_dir} '\
            f'"{url}"'
    return query.replace('$','\\$')

In [52]:
# with open("download.sh", "w") as f:  # writing
with open("download.sh", "a") as f:  # appending
    for i,r in result_df.iterrows():
        f.write(download_data(r['data'])+';sleep 30\n')

In [ ]:
# run download.sh in bash